# <font color=#0404B4>**SEMINAR - PHÂN TÍCH DỮ LIỆU THÔNG MINH** </font>

### <font color=#0404B4> Bảng thành viên nhóm </font>
|<center><div style="width:150px">MSSV</div><center>|<center><div style="width:290px">Họ và tên</div><center>|
|---------- |:-------------:|
| <center>20120032<center>  | <center>Phan Trường An<center> |
| <center>20120037<center>  | <center>Trần Thị Minh Anh<center> |
| <center>20120041<center>  | <center>Trần Kim Bảo<center> |
| <center>20120053<center>  |   <center>Nguyễn Thành Đạt<center>   | 
| <center>20120071<center>  |<center>Nguyễn Thị Bích Hà<center> |
| <center>20120095<center>  | <center>Ngô Quang Hưng<center> |

# <font color=#0404B4>Import thư viện</font>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

<a name="2"></a>
# <font color=#0404B4> A. Tiền xử lý dữ liệu</font>


### 1. Đọc dữ liệu

In [2]:
item_category_df=pd.read_csv("item_categories.csv")
item_category_df.shape

(84, 2)

In [3]:
items_df=pd.read_csv("items.csv")
items_df.shape

(22170, 3)

In [4]:
sales_train_df=pd.read_csv("sales_train.csv")
sales_train_df.shape

(2935849, 6)

In [5]:
sample_submission_df=pd.read_csv("sample_submission.csv")
sample_submission_df.shape

(214200, 2)

In [6]:
shops_df=pd.read_csv("shops.csv")
shops_df.shape

(60, 2)

In [7]:
test_df=pd.read_csv("test.csv")
test_df.shape

(214200, 3)

### 2. Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?
- `item_category_df`: Mỗi dòng là thông tin về danh mục sản phẩm gồm: tên danh mục sản phẩm và id của danh mục sản phẩm đó.
- `items_df`: Mỗi dòng là thông tin về sản phẩm gồm: tên sản phẩm, id của sản phẩm và id của danh mục chứa sản phẩm đó.
- `sales_train_df`: Mỗi dòng là thông tin về bán hàng gồm: ngày bán hàng, thuộc kì bán nào(1 kì ở đây tương ứng với 1 tháng), id của cửa hàng bán sản phẩm đó, id của sản phẩm bán, thông tin về giá của sản phẩm và số lượng bán sản phẩm đó trong 1 ngày.
- `sample_submission_df`: Hướng dẫn định dạng nộp bài gồm các thông tin: ID(thứ tự) dự đoán, số lượng sản phẩm bán ra trong tháng.
- `shops_df`: Mỗi dòng là thông tin về cửa hàng gồm: tên cửa hàng và id của cửa hàng đó
- `test_df`: Chứa dữ liệu kiểm tra cho mô hình dự đoán gồm ID dự đoán, id cửa hàng và id sản phẩm.

### 3. Có dữ liệu bị trùng lặp ở các dataframe không

In [8]:
def check_duplicated(dataframe):
    duplicated_rows = dataframe.duplicated()

    # In các hàng bị lặp lại
    if len(dataframe[duplicated_rows])>0:
        print('Có dòng dữ liệu bị lặp')
        display(dataframe[duplicated_rows])
    else: 
        print('Không có dòng dữ liệu nào bị lặp')


Dataframe `item_category_df`

In [9]:
check_duplicated(item_category_df)

Không có dòng dữ liệu nào bị lặp


Dataframe `items_df`

In [10]:
check_duplicated(items_df)

Không có dòng dữ liệu nào bị lặp


Dataframe `sales_train_df`

In [11]:
check_duplicated(sales_train_df)

Có dòng dữ liệu bị lặp


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
76962,05.01.2013,0,54,20130,149.0,1.0
1435367,23.02.2014,13,50,3423,999.0,1.0
1496766,23.03.2014,14,21,3423,999.0,1.0
1671873,01.05.2014,16,50,3423,999.0,1.0
1866340,12.07.2014,18,25,3423,999.0,1.0
2198566,31.12.2014,23,42,21619,499.0,1.0


Xóa luôn các mẫu dữ liệu bị trùng

In [12]:
sales_train_df.drop_duplicates(inplace=True)

Dataframe `sample_submission_df`

In [13]:
check_duplicated(sample_submission_df)

Không có dòng dữ liệu nào bị lặp


Dataframe `shops_df`

In [14]:
check_duplicated(shops_df)

Không có dòng dữ liệu nào bị lặp


Dataframe `test_df`

In [15]:
check_duplicated(test_df)

Không có dòng dữ liệu nào bị lặp


<h3> 4. Mỗi cột có ý nghĩa gì?</h3>

|STT|Tên cột dữ liệu | <center>Mô tả<center> | Đơn vị |
|--|:------:| ---| --- |
|1|item_category_name|<center>Tên danh mục sản phẩm<center>|
|2|item_category_id| <center>Mã định danh của danh mục sản phẩm<center>|
|3|item_name|<center>Tên sản phẩm<center>| 
|4|item_id| <center>Mã định danh sản phẩm<center>||
|5|date|<center>Ngày bán hàng<center>|| 
|6|date_block_num|<center>Thứ tự kỳ bán hàng *(bắt đầu từ 0)*<center>|
|7|shop_id|<center>Mã định danh cửa hàng<center>| |
|8|item_price|<center>Giá sản phẩm<center>|<center>Rúp<center>|
|9|item_cnt_day|<center>Số lượng bán ra/ số lượng mất trong `1 ngày`<center>|<center>Sản phẩm <center>|
|10|ID|<center>Mã định danh<center>| |
|11|item_cnt_month|<center>Số lượng bán ra/ số lượng mất trong `1 tháng`<center>|<center>Sản phẩm <center>|
|12|shop_name|<center>Tên cửa hàng<center>| |


### 5. Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp?



Dataframe `item_category_df`

In [16]:
item_category_df.columns

Index(['item_category_name', 'item_category_id'], dtype='object')

In [17]:
item_category_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   item_category_name  84 non-null     object
 1   item_category_id    84 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.4+ KB


Dataframe `items_df`

In [18]:
items_df.columns

Index(['item_name', 'item_id', 'item_category_id'], dtype='object')

In [19]:
items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22170 entries, 0 to 22169
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   item_name         22170 non-null  object
 1   item_id           22170 non-null  int64 
 2   item_category_id  22170 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 519.7+ KB


Dataframe `sales_train_df`

In [20]:
sales_train_df.columns

Index(['date', 'date_block_num', 'shop_id', 'item_id', 'item_price',
       'item_cnt_day'],
      dtype='object')

In [21]:
sales_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2935843 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 156.8+ MB


Dataframe `sample_submission_df`

In [22]:
sample_submission_df.columns

Index(['ID', 'item_cnt_month'], dtype='object')

In [23]:
sample_submission_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214200 entries, 0 to 214199
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   ID              214200 non-null  int64  
 1   item_cnt_month  214200 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 3.3 MB


Dataframe `shops_df`

In [24]:
shops_df.columns

Index(['shop_name', 'shop_id'], dtype='object')

In [25]:
shops_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   shop_name  60 non-null     object
 1   shop_id    60 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB


Dataframe `test_df`

In [26]:
test_df.columns

Index(['ID', 'shop_id', 'item_id'], dtype='object')

In [27]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214200 entries, 0 to 214199
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   ID       214200 non-null  int64
 1   shop_id  214200 non-null  int64
 2   item_id  214200 non-null  int64
dtypes: int64(3)
memory usage: 4.9 MB



##### Cột `date` của dataframe `sales_train_df` đang có kiểu dữ liệu `object`. Thực hiện xử lý đưa về đúng kiểu dữ liệu `datime64[ns]`.

In [28]:
sales_train_df.date=pd.to_datetime(sales_train_df.date,format="%d.%m.%Y")

In [29]:
sales_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2935843 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype         
---  ------          -----         
 0   date            datetime64[ns]
 1   date_block_num  int64         
 2   shop_id         int64         
 3   item_id         int64         
 4   item_price      float64       
 5   item_cnt_day    float64       
dtypes: datetime64[ns](1), float64(2), int64(3)
memory usage: 156.8 MB



### 7. Với mỗi cột, các giá trị được phân bố như thế nào?

- Nhận xét:
    - Các dataframe như: `item_category_df`, `items_df`, `sample_submission_df`, `shops_df` và `test_df`, là những dataframe chứa thông tin chi tiết đối đối với dataframe `sales_train_df`. Nên sẽ chỉ tập trung xem phân bố dữ liệu của dataframe `sales_train_df`
    - Với các dữ liệu trong dataframe `sales_train_df` chỉ có dạng số.

#### Cột có kiểu dữ liệu dạng số, và phân bố:


In [30]:
numerical_cols = sales_train_df[sales_train_df.select_dtypes(exclude=['object']).columns]

def missing_ratio(c):
    return  c.isna().sum()/len(c) * 100

def min(c):
    return c.min()

def max(c):
    return c.max()

nume_col_profiles_df = numerical_cols.agg([missing_ratio, min, max])
nume_col_profiles_df

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.0
min,2013-01-01 00:00:00,0.0,0.0,0.0,-1.0,-22.0
max,2015-10-31 00:00:00,33.0,59.0,22169.0,307980.0,2169.0


- Nhận xét:
    - `date`: dữ liệu bán hàng được thu thập từ ngày 01-01-2013 đến ngày 31-10-2015.
    - `date_block_num`: Chia dữ liệu thành 34 kì (1 kì=1 tháng)
    - `shop_id`: ID cửa hàng có giá trị nhỏ nhất là 0, lớn nhất là 59.
    - `item_id`: ID sản phẩm có giá trị nhỏ nhất là 0, lớn nhất là 22169.
    - `item_price`: Sản phẩm có giá tiền nhỏ nhất -1, lớn nhất là 307980 (Rúp).
    - `item_cnt_day`: Số lượng sản phẩm đó được bán ra. Số lượng sản phẩm bị mất lớn nhất là 22 sản phẩm, có thể nhiều lý do như hư hỏng, thất lạc trong quá trình vận chuyển,... Số lượng sản phẩm được bán ra lớn nhất là 2169 sản phẩm. 

###  8. Có cột nào cần phải tiền xử lý hay không?


Kiểm tra thử số lượng `item_price` có giá bị âm

In [31]:
sales_train_df[sales_train_df.item_price<0]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
484683,2013-05-15,4,32,2973,-1.0,1.0


Với chỉ 1 giá sản phẩm bị âm nên ta có thể thực hiện loại xóa dữ liệu đó.

In [32]:
index_drop=sales_train_df[sales_train_df.item_price<0].index
sales_train_df.drop(index_drop)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1.0
1,2013-01-03,0,25,2552,899.00,1.0
2,2013-01-05,0,25,2552,899.00,-1.0
3,2013-01-06,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,2015-10-10,33,25,7409,299.00,1.0
2935845,2015-10-09,33,25,7460,299.00,1.0
2935846,2015-10-14,33,25,7459,349.00,1.0
2935847,2015-10-22,33,25,7440,299.00,1.0


<a name="2"></a>
# <font color=#0404B4> B. Khám phá dữ liệu</font>
